In [4]:
import os

In [5]:
path = "/Users/kadakia/anaconda/bin"
path_2 = "/Users/kadakia/anaconda"

# Check current working directory.
retval = os.getcwd()
print("Current working directory %s" % retval)

# Now change the directory
os.chdir( path )

# Check current working directory.
retval = os.getcwd()

print("Directory changed successfully %s" % retval)

Current working directory /Users/kadakia/anaconda
Directory changed successfully /Users/kadakia/anaconda/bin


In [6]:
# Check current working directory.
retval = os.getcwd()
print("Current working directory %s" % retval)

# Now change the directory
os.chdir(path_2)

# Check current working directory.
retval = os.getcwd()

print("Directory changed successfully %s" % retval)

Current working directory /Users/kadakia/anaconda/bin
Directory changed successfully /Users/kadakia/anaconda


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# concurrency
import multiprocessing

# regex = regular expressions
import glob
import re
import nltk
import gensim.models.word2vec as w2v

# dimension reduction
import sklearn.manifold

In [69]:
raw_corpus = ["Human machine interface",
             "A survey of user opinion, so you can dream big",
             "The EPS user interface and New York",
             "System and human system engineering",
             "The biggest city is New York, dream big"]
#             "Relation of user perceived response time to error measurement",
#             "The generation of random binary unordered trees",
#             "The intersection graph of paths in trees",
#             "Graph minors IV Widths of trees and well quasi ordering",
#             "Graph minors A survey"

In [23]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))

# Lowercase each string/title, split it by white space and filter out stopwords
# Nested list comprehension!
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]
print(texts)

[['human', 'machine', 'interface'], ['survey', 'user', 'opinion'], ['eps', 'user', 'interface', 'new', 'york'], ['system', 'human', 'system', 'engineering'], ['biggest', 'city', 'is', 'new', 'york']]


In [24]:
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(14 unique tokens: ['human', 'machine', 'interface', 'survey', 'user']...)


In [25]:
print(dictionary.token2id)

{'human': 0, 'machine': 1, 'interface': 2, 'survey': 3, 'user': 4, 'opinion': 5, 'eps': 6, 'new': 7, 'york': 8, 'system': 9, 'engineering': 10, 'biggest': 11, 'city': 12, 'is': 13}


In [26]:
bow_corpus = [dictionary.doc2bow(text) for text in texts]
print(bow_corpus) # List of lists of tuples

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1), (5, 1)], [(2, 1), (4, 1), (6, 1), (7, 1), (8, 1)], [(0, 1), (9, 2), (10, 1)], [(7, 1), (8, 1), (11, 1), (12, 1), (13, 1)]]


In [27]:
print(dictionary[0])
print(dictionary[1])
print(dictionary[2])

human
machine
interface


In [28]:
tfidf = models.TfidfModel(bow_corpus, normalize = True)

In [79]:
# weight = term frequency x log (standard is base 2) of inverse document frequency
# then normalize vector at the end
# if word doesn't appear in document, its weight is zero
# if word appears in every document, its weight is zero
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.4434518002301797), (1, 0.7789101371437052), (2, 0.4434518002301797)]
[(3, 0.6559486886294514), (4, 0.37344696513776354), (5, 0.6559486886294514)]
[(2, 0.37568534995939296), (4, 0.37568534995939296), (6, 0.6598803462019109), (7, 0.37568534995939296), (8, 0.37568534995939296)]
[(0, 0.24673728071874365), (9, 0.8667736564079552), (10, 0.4333868282039776)]
[(7, 0.2980686310151986), (8, 0.2980686310151986), (11, 0.5235488459890668), (12, 0.5235488459890668), (13, 0.5235488459890668)]


In [30]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # SVD vs cosine similarity....

In [31]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

[(0, 0.47215037167268659), (1, -0.60079910933792469)]
[(0, 0.3506313327530497), (1, 0.22800794016782902)]
[(0, 0.79549292051019371), (1, 0.14763755283166582)]
[(0, 0.16230157766407116), (1, -0.72361975631637943)]
[(0, 0.55971643152157657), (1, 0.36397143868259363)]


In [35]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kadakia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kadakia/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [32]:
words_types = []

for doc in raw_corpus:
    tokens = nltk.word_tokenize(doc)
    tagged = nltk.pos_tag(tokens) # pos = part of speech
    words_types.append(tagged)
    
print(words_types) # Better results with periods at the ends of sentences?

[[('Human', 'NNP'), ('machine', 'NN'), ('interface', 'NN')], [('A', 'DT'), ('survey', 'NN'), ('of', 'IN'), ('user', 'JJ'), ('opinion', 'NN')], [('The', 'DT'), ('EPS', 'NNP'), ('user', 'NN'), ('interface', 'NN'), ('and', 'CC'), ('New', 'NNP'), ('York', 'NNP')], [('System', 'NN'), ('and', 'CC'), ('human', 'JJ'), ('system', 'NN'), ('engineering', 'NN')], [('The', 'DT'), ('biggest', 'JJS'), ('city', 'NN'), ('is', 'VBZ'), ('New', 'NNP'), ('York', 'NNP')]]


In [171]:
tokens = nltk.word_tokenize("I am learning very quickly")
tagged = nltk.pos_tag(tokens)
print(tagged)

[('I', 'PRP'), ('am', 'VBP'), ('learning', 'VBG'), ('very', 'RB'), ('quickly', 'RB')]


In [33]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Tokenize into sentences, not individual words!
raw_sentences = tokenizer.tokenize("I am good. You are bad.")

for sentence in raw_sentences:
    print(sentence)

I am good.
You are bad.


In [120]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to
[n

[nltk_data]    |   Unzipping corpora/sentiwordnet.zip.
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/sentence_polarity.zip.
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/shakespeare.zip.
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/sinica_treebank.zip.
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/smultron.zip.
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/state_union.zip.
[nltk_data]    | Downloading package stopwords to
[nltk_data]    |     /Users/kadakia/nltk_data...
[nltk_data]    |   Unzipping corpora/stopwords.zip.
[nltk

True

In [34]:
# Convert string to words?
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# from nltk.stem import PorterStemmer

stop_words = stopwords.words('English')
print(stop_words)
lemmatizer = WordNetLemmatizer()
# ps = PorterStemmer()

def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z0-9]", " ", raw)
    words = clean.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w)>2]
    if len(words) > 0:
        words[0] = words[0].lower()
    return words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [70]:
corpus_raw = ""

for title in raw_corpus:
    corpus_raw += title + '. '
    
print(corpus_raw)

Human machine interface. A survey of user opinion, so you can dream big. The EPS user interface and New York. System and human system engineering. The biggest city is New York, dream big. 


In [71]:
raw_sentences = tokenizer.tokenize(corpus_raw)

for sentence in raw_sentences:
    print(sentence)

Human machine interface.
A survey of user opinion, so you can dream big.
The EPS user interface and New York.
System and human system engineering.
The biggest city is New York, dream big.


In [72]:
sentences = []

for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        s = sentence_to_wordlist(raw_sentence)
        sentences.append(s)
        
print(sentences) # "The" remains because it was capitalized

[['human', 'machine', 'interface'], ['survey', 'user', 'opinion', 'dream', 'big'], ['the', 'EPS', 'user', 'interface', 'New', 'York'], ['system', 'human', 'system', 'engineering'], ['the', 'biggest', 'city', 'New', 'York', 'dream', 'big']]


In [73]:
print(len(sentences))

token_count = sum([len(sentence) for sentence in sentences])
print(token_count)

5
25


In [74]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

phrases = Phrases(sentences, min_count = 1, threshold = 2)
bigram = Phraser(phrases)

In [75]:
import pickle
pickle.dump(bigram, open("bigram.pkl", "wb"))

In [78]:
sentences_bigram = list(bigram[sentences])

sample = ['the', 'EPS', 'user', 'interface', 'New', 'York', 'dream', 'big']

print(bigram[sample])

['the', 'EPS', 'user', 'interface', 'New_York', 'dream_big']


# Truncated SVD

In [107]:
from gensim import matutils

num_terms = len(corpus_tfidf.obj.idfs)

tfidf_dense = matutils.corpus2dense(corpus_tfidf, num_terms)

print(tfidf_dense)
print(type(tfidf_dense))
print(tfidf_dense.shape)

[[ 0.40824831  0.          0.          0.21821789]
 [ 0.81649661  0.          0.          0.        ]
 [ 0.40824831  0.          0.40824831  0.        ]
 [ 0.          0.66666669  0.          0.        ]
 [ 0.          0.33333334  0.40824831  0.        ]
 [ 0.          0.66666669  0.          0.        ]
 [ 0.          0.          0.81649661  0.        ]
 [ 0.          0.          0.          0.87287158]
 [ 0.          0.          0.          0.43643579]]
<class 'numpy.ndarray'>
(9, 4)


In [108]:
U, s, V = np.linalg.svd(tfidf_dense)

print(U.shape, V.shape, s.shape)

S = np.zeros((9, 4))
S[:4, :4] = np.diag(s)

print(S)
#print(np.dot(U, np.dot(S, V)))

# np.allclose(tfidf_dense, np.dot(U, np.dot(S, V)))

(9, 9) (4, 4) (4,)
[[ 1.10754395  0.          0.          0.        ]
 [ 0.          1.02639556  0.          0.        ]
 [ 0.          0.          0.97288859  0.        ]
 [ 0.          0.          0.          0.87940121]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


In [117]:
s_2 = s[0:2]

S_2 = np.zeros((9, 4))
S_2[:2, :2] = np.diag(s_2)

#print(S_2)
print(np.dot(U, np.dot(S_2, V)))

np.set_printoptions(formatter={'float_kind':'{:f}'.format})

print(np.dot(U, np.dot(S_2, V)))

[[  2.91822879e-01  -6.49130119e-02   1.21441119e-01   2.73178172e-01]
 [  4.08248349e-01  -3.30268670e-08   2.42882228e-01   3.28138519e-01]
 [  3.25565289e-01   1.64069237e-01   3.25565246e-01   1.64069261e-01]
 [ -2.62791214e-08   3.33333352e-01   2.67923971e-01  -1.98312506e-01]
 [  1.21441110e-01   3.30735948e-01   3.38086138e-01  -9.91562561e-02]
 [ -2.62791214e-08   3.33333352e-01   2.67923971e-01  -1.98312506e-01]
 [  2.42882249e-01   3.28138520e-01   4.08248287e-01   1.02841180e-08]
 [  3.50794831e-01  -2.59652024e-01  -3.35134889e-09   4.36435688e-01]
 [  1.75397415e-01  -1.29826012e-01  -1.67567445e-09   2.18217844e-01]]
[[0.291823 -0.064913 0.121441 0.273178]
 [0.408248 -0.000000 0.242882 0.328139]
 [0.325565 0.164069 0.325565 0.164069]
 [-0.000000 0.333333 0.267924 -0.198313]
 [0.121441 0.330736 0.338086 -0.099156]
 [-0.000000 0.333333 0.267924 -0.198313]
 [0.242882 0.328139 0.408248 0.000000]
 [0.350795 -0.259652 -0.000000 0.436436]
 [0.175397 -0.129826 -0.000000 0.218218

In [94]:
C = np.array([[1,0,1,1,0],[0,1,1,0,0],[1,0,0,0,0],[0,0,0,1,1],[0,0,0,1,0],[0,0,0,0,1]])
print(C.shape)

(6, 5)


In [99]:
U, s, V = np.linalg.svd(C.T)
print(U)
print(s)
print(V)

[[  4.40347480e-01  -2.96174360e-01  -5.69497581e-01   5.77350269e-01
   -2.46402144e-01]
 [  1.29346349e-01  -3.31450692e-01   5.87021697e-01   1.11022302e-16
   -7.27197008e-01]
 [  4.75530263e-01  -5.11115242e-01   3.67689978e-01   1.11022302e-16
    6.14358412e-01]
 [  7.03020318e-01   3.50572409e-01  -1.54905878e-01  -5.77350269e-01
   -1.59788154e-01]
 [  2.62672838e-01   6.46746769e-01   4.14591704e-01   5.77350269e-01
    8.66139898e-02]]
[ 2.16250096  1.59438237  1.27529025  1.          0.39391525]
[[  7.48623048e-01   2.79711603e-01   2.03628802e-01   4.46563110e-01
    3.25095956e-01   1.21467154e-01]
 [ -2.86453991e-01  -5.28459139e-01  -1.85761186e-01   6.25520701e-01
    2.19879758e-01   4.05640944e-01]
 [ -2.79711603e-01   7.48623048e-01  -4.46563110e-01   2.03628802e-01
   -1.21467154e-01   3.25095956e-01]
 [ -3.88578059e-16   3.33066907e-16   5.77350269e-01   5.55111512e-17
   -5.77350269e-01   5.77350269e-01]
 [  5.28459139e-01  -2.86453991e-01  -6.25520701e-01  -1.85

In [103]:
s_2 = s[0:1]

S_2 = np.zeros((5, 6))
S_2[:1, :1] = np.diag(s_2)

print(S_2)
print(np.dot(U, np.dot(S_2, V)))

[[ 2.16250096  0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]
[[ 0.71287768  0.26635589  0.1939059   0.42524055  0.30957323  0.11566732]
 [ 0.20939855  0.07823858  0.05695734  0.12490888  0.09093311  0.03397577]
 [ 0.76983502  0.28763713  0.20939855  0.45921632  0.33430744  0.12490888]
 [ 1.13811823  0.42524055  0.30957323  0.67890191  0.49423757  0.18466434]
 [ 0.42524055  0.15888466  0.11566732  0.25366136  0.18466434  0.06899702]]
